<a href="https://colab.research.google.com/github/ViennaRNA/RNAdeep/blob/main/windows_lstms_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Check GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
#Mount Drive
#TODO Change to wget
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
#os.chdir("drive/My Drive/")
os.listdir()


In [ ]:
!cp -r drive/My\ Drive/data/interim/10000_length70 .
!cp drive/My\ Drive/models/*py .
#!unzip 100000_length70.zip

In [ ]:
from sliding_window import basic_window,conv_window
from metrics import mcc,f1,sensitivity
from data_generators import DataFromInterimWindows
import numpy as np
from tensorflow.keras.callbacks import CSVLogger,ModelCheckpoint,Callback
import os
import statistics
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.01)

directory = "10000_length70"
batch_size = 128
size = 80000
window_size = 15

m = conv_window(window_size) #use basic_window or conv_window here 
csvname = "window__%i_%i.csv" % (batch_size,window_size)
csv_logger = CSVLogger(csvname, append=True, separator=';')
m.compile(optimizer=opt,loss="binary_crossentropy", metrics=["acc",mcc,f1])
train_generator = DataFromInterimWindows(batch_size,os.path.join(directory,"train"),window_size)
val_generator = DataFromInterimWindows(batch_size,os.path.join(directory,"val"),window_size)
m.fit(x=train_generator, validation_data = val_generator,epochs=100,shuffle = True,verbose=1,callbacks=[csv_logger])

In [ ]:
from lstm_models import blstm
from metrics import mcc,f1,sensitivity
from data_generators import DataFromInterimFile
import numpy as np
from tensorflow.keras.callbacks import CSVLogger,ModelCheckpoint,Callback
import os
import statistics
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.01)

directory = "100000_length70"
size = 80000
batch_size = 128

time_callback = TimeHistory()
layer=1
neurons=80
m = blstm(5,80)
csvname = "blstm_%i_%i_%i.csv" % (size,batch_size,window_size)
csv_logger = CSVLogger(csvname, append=True, separator=';')
m.compile(optimizer=opt,loss="binary_crossentropy", metrics=["acc",mcc,f1])
train_generator = DataFromInterimFile(batch_size,os.path.join(directory,"train"))
val_generator = DataFromInterimFile(batch_size,os.path.join(directory,"val"))
m.fit(x=train_generator, validation_data = val_generator,epochs=5,shuffle = True,verbose=1,callbacks=[time_callback ])
